In [35]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
class LinearSVM:
    def __init__(self, C=1.0,lr = 0.01, n_epochs=1000,random_state=42):
        self.C = C
        self.lr = lr
        self.n_epochs = n_epochs
        self.random_state = random_state
        self.w = None
        self.b = None
    def _prepare_labels(self, y):
        y = np.asarray(y)
        y_unique = np.unique(y)
        if set(y_unique.tolist()) == {0,1}:
            return np.where(y == 1,1,-1)
        elif set(y_unique.tolist()) == {-1,1}:
            return y
        else:
            raise ValueError("Label error")
    
    def fit(self,X,y):
        rng = np.random.default_rng(self.random_state)
        X = np.asarray(X, dtype=np.float64)
        y = self._prepare_labels(y)
        N, d = X.shape
        self.w = rng.normal(scale=0.1,size = d)
        self.b = 0.0
        for epoch in range(self.n_epochs):
            scores = X @ self.w + self.b
            margins = y * scores
            
            mask = margins < 1

            grad_w = self.w.copy()

            if np.any(mask):
                grad_w -= self.C * (X[mask] * y[mask][:, None]).sum(axis=0) / N
                grad_b = - self.C * y[mask].sum() / N
            else:
                grad_b = 0.0
            
            self.w -= self.lr * grad_w
            self.b -= self.lr * grad_b
        
        return self
    
    def decision_function(self, X):
        X = np.asarray(X, dtype=np.float64)
        return X @ self.w + self.b
    

    def predict(self, X):
        scores = self.decision_function(X)
        y_pred = np.sign(scores)
        y_pred[y_pred == 0] = 1
        return np.where(y_pred == 1, 1, 0)





In [36]:
df = pd.read_csv("./train.csv")
df.columns

Index(['id', 'survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked'],
      dtype='object')

In [37]:
y = df["survived"]
X = df.drop(columns="survived")
X.shape

(445, 8)

In [38]:
cat_cols = df.select_dtypes(include=["object","category"]).columns.tolist()
num_cols = df.select_dtypes(include=["float64","int64"]).drop(columns="survived").columns.tolist()

In [39]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2, random_state=42, stratify=y)

In [40]:
for col in num_cols:
    median = X_train[col].median()
    X_train[col] = X_train[col].fillna(median)
    X_test[col] = X_test[col].fillna(median)

for col in cat_cols:
    mode = X_train[col].mode(dropna=True)
    if len(mode) > 0:
        fill_value = mode.iloc[0]
    else:
        fill_value = "miss"
    X_train[col] = X_train[col].fillna(fill_value)
    X_test[col] = X_test[col].fillna(fill_value)

X_train_cat = pd.get_dummies(X_train[cat_cols], drop_first=False)
X_test_cat  = pd.get_dummies(X_test[cat_cols],  drop_first=False)

X_train_cat, X_test_cat = X_train_cat.align(
    X_test_cat, join="left", axis=1, fill_value=0
)
X_train_num = X_train[num_cols].astype(float)
X_test_num  = X_test[num_cols].astype(float)

X_train_all = pd.concat([X_train_num, X_train_cat], axis=1)
X_test_all  = pd.concat([X_test_num,  X_test_cat],  axis=1)


In [41]:
mean = X_train_all.mean(axis=0)
std  = X_train_all.std(axis=0)

std_replaced = std.replace(0, 1.0)

X_train_scaled = (X_train_all - mean) / std_replaced
X_test_scaled  = (X_test_all  - mean) / std_replaced

X_train_np = X_train_scaled.values.astype(np.float64)
X_test_np  = X_test_scaled.values.astype(np.float64)

print("X_train shape:", X_train_np.shape)
print("X_test  shape:", X_test_np.shape)


X_train shape: (356, 11)
X_test  shape: (89, 11)


In [42]:
svm = LinearSVM(C=1.0, lr=0.01, n_epochs=1000)
svm.fit(X_train_np, y_train)
y_pred = svm.predict(X_test_np)
print("Test acc:", (y_pred == y_test).mean())

Test acc: 0.7865168539325843
